In [16]:
import torch
from torch import nn
import tinycudann as tcnn
import torch.nn.functional as F
import vren
from einops import rearrange
import numpy as np

In [2]:
L = 4
F = 2
log2_T = 10
N_min = 4
b = 1.25
rgb_act = "Sigmoid"

xyz_encoder = \
    tcnn.NetworkWithInputEncoding(
        n_input_dims=3, n_output_dims=16,
        encoding_config={
            "otype": "Grid",
            "type": "Hash",
            "n_levels": L,
            "n_features_per_level": F,
            "log2_hashmap_size": log2_T,
            "base_resolution": N_min,
            "per_level_scale": b,
            "interpolation": "Linear"
        },
        network_config={
            "otype": "FullyFusedMLP",
            "activation": "ReLU",
            "output_activation": "None",
            "n_neurons": 64,
            "n_hidden_layers": 1,
        }
    )

dir_encoder = \
    tcnn.Encoding(
        n_input_dims=3,
        encoding_config={
            "otype": "SphericalHarmonics",
            "degree": 4,
        },
    )

rgb_net = \
    tcnn.Network(
        n_input_dims=32, n_output_dims=3,
        network_config={
            "otype": "FullyFusedMLP",
            "activation": "ReLU",
            "output_activation": rgb_act,
            "n_neurons": 64,
            "n_hidden_layers": 2,
        }
    )

In [5]:
x = torch.randn((32, 3)).cuda()
gamma_x = xyz_encoder(x)
gamma_x.shape

torch.Size([32, 16])

In [17]:
d = torch.randn((32, 3)).cuda()
d = F.normalize(d, dim=1)
gamma_d = dir_encoder(d)
gamma_d.shape

torch.Size([32, 16])

In [19]:
gamma_d[0]

tensor([  0.2820,   1.2305,  -0.4724,   1.1240,   6.3320,  -2.6621,   0.5693,
         -2.4297,  -0.5767,  14.1641, -16.2031,   4.2344,  -0.6050,   3.8652,
          1.4756, -18.6562], device='cuda:0', dtype=torch.float16,
       grad_fn=<SelectBackward0>)

In [8]:
rgb = rgb_net(torch.cat([gamma_x, gamma_d], 1))

In [9]:
rgb.shape

torch.Size([32, 3])